# Agent-based peptide discovery

In [ ]:
import autogen
import os
import tqdm
import time
import openai
from openai import AsyncOpenAI
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from IPython.display import Markdown, display
os.environ["AUTOGEN_USE_DOCKER"] = "0"
from tqdm import tqdm


In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding

In [ ]:
import os

# Credentials are read from environment variables.
# See the repo root `.env.example` (copy to `.env`, do not commit).

# Optional: Hugging Face token (needed for some gated models).
try:
    from huggingface_hub import login
except Exception:
    login = None

hf_token = os.getenv('HF_TOKEN')
if hf_token and login:
    login(token=hf_token)
elif hf_token and not login:
    print('HF_TOKEN is set but `huggingface_hub` is not installed; skipping login.')
else:
    print('HF_TOKEN not set; skipping Hugging Face login.')

# OpenAI/AutoGen expects OPENAI_API_KEY in the environment.
if not os.getenv('OPENAI_API_KEY'):
    raise RuntimeError('OPENAI_API_KEY is not set. See `.env.example`.')


### Using GPT-4 - Main setup starts here

In [ ]:
device = "cpu"
import transformers
autogen.__version__, openai.__version__, transformers.__version__ #make sure to update both autogen and openai to their newsest versions 
# ('0.2.2', '1.6.0', '4.35.1')

In [ ]:
from llm_config import llm_config
from llm_config import config_list
import agent_functions as func
import agents
import autogen

In [ ]:
def _reset_agents():
    agents.user_proxy.reset()
    #agents.ragproxyagent.reset()
    agents.planner.reset()
    agents.assistant.reset()
    agents.critic.reset()
    #agents.reviewer.reset()
    #agents.coder.reset()
    #agents.executor.reset()
    #sequence_retriever.reset()
#manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)
_reset_agents()

manager_llm_config = {
    "config_list": config_list,
    "seed": 45,
}

groupchat = autogen.GroupChat(
    agents=[agents.user_proxy, agents.planner, agents.assistant, agents.critic, #agents.coder, agents.executor  #sequence_retriever,
               ], messages=[], max_round=150, 
)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config = manager_llm_config, 
                system_message='This agent repeats the following steps: Dynamically selecting a speaker, collecting response, and \
broadcasting the message to the group.')

In [ ]:
# agents.user_proxy.initiate_chat(
#     recipient=manager,
#     message='''
# Can you provide some names of antimicrobial resistant bacteria?
# '''
# )
agents.user_proxy.initiate_chat(
    recipient=manager,
    message='''
For the local file "smorfs_10_50aa.faa" in the workspace containing small open reading frame (smORF)sequences, can you discover AMPs (antimicrobial peptides) that are effective and safe to use against the antimicrobial resistant fungi Fusarium oxysporum?
'''
)


In [ ]:
agents.user_proxy.send(
    recipient=manager,
    message='''Can you fetch verfied AMPs from the database that might be effective against Fusarium oxysporum, and for the predicted results in "amp_mic_tox_hemo.csv" you provided that are effective and safe to use against the antimicrobial resistant fungi Fusarium oxysporum, can you further filter the most promising AMP candidates in "amp_mic_tox_hemo.csv" by getting similarity scores against the verfied AMPs you fetched from the database?
    '''
)